In [ ]:
pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 22.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pennylane as qml
import jax
from jax import numpy as jnp
import optax
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Step 1: Mount the dataset
# Assuming the dataset is available as a CSV file
data_path = "/content/typeII_AGN_metadata.csv"
df = pd.read_csv(data_path)

# Step 2: Target Column and Features
target_column = 'log_bh_mass'
features_columns = [
    'h_beta_flux', 'h_beta_flux_err', 'oiii_5007_flux', 'oiii_5007_flux_err',
    'h_alpha_flux', 'h_alpha_flux_err', 'nii_6584_flux', 'nii_6584_flux_err',
    'log_stellar_sigma', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
    'psfMag_z', 'psfMagErr_u', 'psfMagErr_g', 'psfMagErr_r', 'psfMagErr_i',
    'psfMagErr_z', 'mendel_logM_p50', 'mendel_logM_p16', 'mendel_logM_p84',
    'mendel_logMt_p50', 'mendel_logMt_p16', 'mendel_logMt_p84', 'mendel_logMb_p50',
    'mendel_logMb_p16', 'mendel_logMb_p84', 'mendel_logMd_p50', 'mendel_logMd_p16',
    'mendel_logMd_p84', 'simard_b_t_g', 'simard_e_b_t_g', 'simard_b_t_r',
    'simard_e_b_t_r', 'simard_Rhlg', 'simard_Rhlr', 'simard_Rchl_g',
    'simard_Rchl_r', 'simard_Re', 'simard_e_Re', 'simard_e', 'simard_e_e',
    'simard_nb', 'simard_e_nb', 'simard_PpS', 'simard_Pn4'
]

# Step 3: Handle Missing/NaN values by filling with the mean
df.fillna(df.mean(), inplace=True)

# Separate features and target
X = df[features_columns].values
y = df[target_column].values

# Step 4: Normalize the features
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Step 5: PCA for dimensionality reduction to 4 features
pca = PCA(n_components=4)
X_reduced = pca.fit_transform(X)

# Split the data into 80:20 train-test
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# Convert to JAX arrays
X_train = jnp.array(X_train)
X_test = jnp.array(X_test)
y_train = jnp.array(y_train)
y_test = jnp.array(y_test)

# Step 6: Define the quantum circuit and the QML model
n_wires = 4
dev = qml.device("default.qubit", wires=n_wires)

In [ ]:
@qml.qnode(dev, interface='jax')
def circuit(data, weights):
    """Quantum circuit ansatz"""
    for i in range(n_wires):
        qml.RY(data[i], wires=i)

    for i in range(n_wires):
        qml.RX(weights[i, 0], wires=i)
        qml.RY(weights[i, 1], wires=i)
        qml.RX(weights[i, 2], wires=i)
        qml.CNOT(wires=[i, (i + 1) % n_wires])

    return qml.expval(qml.sum(*[qml.PauliZ(i) for i in range(n_wires)]))

def my_model(data, weights, bias):
    return circuit(data, weights) + bias

# Define loss function
@jax.jit
def loss_fn(params, data, targets):
    predictions = jax.vmap(my_model, in_axes=(0, None, None))(data, params["weights"], params["bias"])
    loss = jnp.mean((targets - predictions) ** 2)
    return loss

# Initialize parameters
weights = jnp.ones([n_wires, 3])
bias = jnp.array(0.0)
params = {"weights": weights, "bias": bias}

# Create the optimizer
opt = optax.adam(learning_rate=0.3)
opt_state = opt.init(params)

# Define the optimization step
@jax.jit
def update_step(params, opt_state, data, targets):
    loss_val, grads = jax.value_and_grad(loss_fn)(params, data, targets)
    updates, opt_state = opt.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return params, opt_state, loss_val

In [ ]:
# Training loop
loss_history = []
num_epochs = 250

for epoch in range(num_epochs):
    params, opt_state, loss_val = update_step(params, opt_state, X_train, y_train)
    loss_history.append(loss_val)
    print(f"Epoch: {epoch}, Loss: {loss_val}")

# Step 7: Evaluate the model
y_pred_train = jax.vmap(my_model, in_axes=(0, None, None))(X_train, params["weights"], params["bias"])
y_pred_test = jax.vmap(my_model, in_axes=(0, None, None))(X_test, params["weights"], params["bias"])


r2 = r2_score(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)

# Calculate the standard deviation and accuracy by error
def calc_std_and_accuracy(y_true, y_pred):
    errors = y_true - y_pred
    std = np.std(errors)
    y_range = np.max(y_true) - np.min(y_true)
    accuracy = (1 - np.abs(errors) / y_range) * 100
    return std, accuracy

# Get the standard deviation and accuracy
std, accuracy = calc_std_and_accuracy(y_test, y_pred_test)

Epoch: 0, Loss: 0.3773617148399353
Epoch: 1, Loss: 0.377393513917923
Epoch: 2, Loss: 0.37727296352386475
Epoch: 3, Loss: 0.37723076343536377
Epoch: 4, Loss: 0.37714648246765137
Epoch: 5, Loss: 0.37730419635772705
Epoch: 6, Loss: 0.3772878050804138
Epoch: 7, Loss: 0.3772973120212555
Epoch: 8, Loss: 0.3772176206111908
Epoch: 9, Loss: 0.3770988881587982
Epoch: 10, Loss: 0.37716954946517944
Epoch: 11, Loss: 0.37706074118614197
Epoch: 12, Loss: 0.3770756423473358
Epoch: 13, Loss: 0.37723517417907715
Epoch: 14, Loss: 0.37724682688713074
Epoch: 15, Loss: 0.3773193061351776
Epoch: 16, Loss: 0.3771543800830841
Epoch: 17, Loss: 0.3773341774940491
Epoch: 18, Loss: 0.3773270845413208
Epoch: 19, Loss: 0.37720823287963867
Epoch: 20, Loss: 0.3771560490131378
Epoch: 21, Loss: 0.37692710757255554
Epoch: 22, Loss: 0.3772311210632324
Epoch: 23, Loss: 0.37731263041496277
Epoch: 24, Loss: 0.377319872379303
Epoch: 25, Loss: 0.37727904319763184
Epoch: 26, Loss: 0.3772953450679779
Epoch: 27, Loss: 0.377202451

In [ ]:
# Print the error metrics

print(f"R^2: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"Standard Deviation of Errors: {std:.4f}")

# Calculate the range of y_train
y_train_range = np.max(y_train) - np.min(y_train)

# Calculate accuracy by MAE, MSE, and RMSE
accuracy_by_mae = (1 - mae / y_train_range) * 100
accuracy_by_mse = (1 - mse / y_train_range) * 100
accuracy_by_rmse = (1 - rmse / y_train_range) * 100

# Print the accuracy metrics
print(f"Accuracy by MAE: {accuracy_by_mae:.2f}%")
print(f"Accuracy by MSE: {accuracy_by_mse:.2f}%")
print(f"Accuracy by RMSE: {accuracy_by_rmse:.2f}%")



R^2: 0.0824
MAE: 0.4817
MSE: 0.3715
RMSE: 0.6095
Standard Deviation of Errors: 0.6094
Accuracy by MAE: 90.45%
Accuracy by MSE: 92.63%
Accuracy by RMSE: 87.91%
